In [1]:
import os

from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_io as tfio

In [2]:
@tf.function
def load_wav_16k_mono(filename):
    """ Load a WAV file, convert it to a float tensor, resample to 16 kHz single-channel audio. """
    file_contents = tf.io.read_file(filename)
    wav, sample_rate = tf.audio.decode_wav(
          file_contents,
          desired_channels=1)
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

In [3]:
yamnet_model_handle = 'https://tfhub.dev/google/yamnet/1'
yamnet_model = hub.load(yamnet_model_handle)

2022-07-14 22:00:20.435203: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-14 22:00:20.498085: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-07-14 22:00:20.498789: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-07-14 22:00:20.499545: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN

In [4]:
testing_wav_data = load_wav_16k_mono("Downloads/ml-stuttering-events-dataset/clips/FluencyBank/178/FluencyBank_178_8.wav")   

pd_data = pd.read_csv("yamnet_data.csv")

my_classes = ['Humming', 'Speech']

filenames = pd_data['filename']
targets = pd_data['target']
folds = pd_data['fold']

main_ds = tf.data.Dataset.from_tensor_slices((filenames, targets, folds))

2022-07-14 22:00:38.041624: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: AVX2 FMA


In [5]:
def load_wav_for_map(filename, label, fold):
  return load_wav_16k_mono(filename), label, fold

main_ds = main_ds.map(load_wav_for_map)

In [6]:
def extract_embedding(wav_data, label, fold):
  ''' run YAMNet to extract embedding from the wav data '''
  scores, embeddings, spectrogram = yamnet_model(wav_data)
  num_embeddings = tf.shape(embeddings)[0]
  return (embeddings,
            tf.repeat(label, num_embeddings),
            tf.repeat(fold, num_embeddings))

# extract embedding
main_ds = main_ds.map(extract_embedding).unbatch()

In [7]:
cached_ds = main_ds.cache()
train_ds = cached_ds.filter(lambda embedding, label, fold: fold < 89)
val_ds = cached_ds.filter(lambda embedding, label, fold: 89 <= fold and fold <= 97)
test_ds = cached_ds.filter(lambda embedding, label, fold: 98 <= fold)

# remove the folds column now that it's not needed anymore
remove_fold_column = lambda embedding, label, fold: (embedding, label)

train_ds = train_ds.map(remove_fold_column)
val_ds = val_ds.map(remove_fold_column)
test_ds = test_ds.map(remove_fold_column)

train_ds = train_ds.cache().shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.cache().batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.cache().batch(32).prefetch(tf.data.AUTOTUNE)

In [8]:
my_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(1024), dtype=tf.float32,
                          name='input_embedding'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(len(my_classes))
], name='my_model')

my_model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               524800    
                                                                 
 dense_1 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 525,826
Trainable params: 525,826
Non-trainable params: 0
_________________________________________________________________


In [9]:
my_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                 optimizer="adam",
                 metrics=['accuracy'])

callback = tf.keras.callbacks.EarlyStopping(monitor='loss',
                                            patience=3,
                                            restore_best_weights=True)
                                            
history = my_model.fit(train_ds,
                       epochs=20,
                       validation_data=val_ds,callbacks=callback)

Epoch 1/20
272/272 [==============================] - 87s 294ms/step - loss: 0.5648 - accuracy: 0.7219 - val_loss: 0.5405 - val_accuracy: 0.8150
Epoch 2/20
272/272 [==============================] - 2s 7ms/step - loss: 0.5461 - accuracy: 0.7242 - val_loss: 0.4960 - val_accuracy: 0.8147
Epoch 3/20
272/272 [==============================] - 2s 7ms/step - loss: 0.5318 - accuracy: 0.7312 - val_loss: 0.5293 - val_accuracy: 0.8150
Epoch 4/20
272/272 [==============================] - 2s 7ms/step - loss: 0.5313 - accuracy: 0.7323 - val_loss: 0.5337 - val_accuracy: 0.8150
Epoch 5/20
272/272 [==============================] - 2s 8ms/step - loss: 0.5096 - accuracy: 0.7440 - val_loss: 0.5568 - val_accuracy: 0.8147
Epoch 6/20
272/272 [==============================] - 2s 7ms/step - loss: 0.4993 - accuracy: 0.7510 - val_loss: 0.5628 - val_accuracy: 0.8140
Epoch 7/20
272/272 [==============================] - 2s 7ms/step - loss: 0.4920 - accuracy: 0.7576 - val_loss: 0.5297 - val_accuracy: 0.8134
Epo

In [10]:
class ReduceMeanLayer(tf.keras.layers.Layer):
  def __init__(self, axis=0, **kwargs):
    super(ReduceMeanLayer, self).__init__(**kwargs)
    self.axis = axis

  def call(self, input):
    return tf.math.reduce_mean(input, axis=self.axis)

In [11]:
saved_model_path = './stuttering_yamnet'

input_segment = tf.keras.layers.Input(shape=(), dtype=tf.float32, name='audio')
embedding_extraction_layer = hub.KerasLayer(yamnet_model_handle,
                                            trainable=False, name='yamnet')
_, embeddings_output, _ = embedding_extraction_layer(input_segment)
serving_outputs = my_model(embeddings_output)
serving_outputs = ReduceMeanLayer(axis=0, name='classifier')(serving_outputs)
serving_model = tf.keras.Model(input_segment, serving_outputs)
serving_model.save(saved_model_path, include_optimizer=False)   

INFO:tensorflow:Assets written to: ./stuttering_yamnet/assets


INFO:tensorflow:Assets written to: ./stuttering_yamnet/assets


In [13]:
reloaded_model = tf.saved_model.load('./stuttering_yamnet')

testing_wav_data = load_wav_16k_mono("Downloads/ml-stuttering-events-dataset/clips/FluencyBank/093/FluencyBank_093_"+str(1)+".wav")


In [14]:
reloaded_results = reloaded_model(testing_wav_data)
stutter = my_classes[tf.argmax(reloaded_results)]
print(f'The main sound is: {stutter}')

The main sound is: Speech
